# Musterlösungen

Zur Berechnung der Kündigungsfristen im Mietrecht, muss ein Kalender zur
Verfügung stehen, welcher die Feiertage und die Wochenenden kennt. Die
`datetime` Library ist eine Python Standard Library. Mit ihr werden die
Kalenderdaten inklusive der Wochentage zur Verfügung gestellt. In dieser Library
nicht integriert sind die Feiertage. Diese müssen mit einer externen Library
geladen werden. Die Library `holidays` stellt (unter anderem) die Feiertage in
der Schweiz zur Verfügung. Sie berücksichtigt auch die kantonalen Feiertage.  
Installiert wird die Library nach dem Starten einer Python Virtual Environment
mit dem Befehl (auf Google Colab ist die Library bereits installiert und dieser
Schritt daher nicht nötig):

```bash
pip install holidays
```

Eine Anleitung für die Verwendung der Library `holidays` wird durch die
[Dokumentation](https://holidays.readthedocs.io/en/latest/#)
zur Verfügung gestellt.

In [1]:
import datetime           # Python standard library
import holidays           # Third-party library

## Kündigungsfristen im Mietrecht

Art. 266c OR regelt die Kündigungsfrist von gemieteten Wohnungen. Die Frist
beträgt drei Monate auf die ortsübliche Kündigungstermine.  
Kündigungen sind Empfangsbedürftige Erklärungen. Das bedeutet, dass für die
Berechnung der Fristen auf den Zeitpunkt des Empfangs der Kündigung abzustellen
ist.

Für die Berechnung, wann eine Kündigung spätestens der Schweizerischen Post
übergeben werden muss, damit die Kündungsfrist eingehalten wird, müssen die
ortsüblichen Kündigungstermine bekannt sein. Im Kanton Zürich ist dabei zu
unterscheiden zwischen der Stadt Zürich und dem Rest des Kantons. In der Stadt
Zürich sind 31. März und der 30. September, im Rest des Kantons der 31. März,
der 30. Juni und der 30. September ordetnliche Kündigungstermine.
Ausserdem muss die Zustellungsdauer
der Post bekannt sein. Für eingeschriebene Briefe beträgt diese in der Regel 1
Tag. 

## Python Funktionen für die Datumsberechnung

### `datetime` Library

Die Python datetime Library stellt die Klasse `datetime` zur Verfügung. Mit
dieser Klasse kann einer Varibeln ein Datum zugewiesen werden. Ausserdem wird
jedem Datum ein Wochentag zugewiesen. Die Wochentage sind von 0 (Montag) bis 6
(Sonntag) nummeriert. Die Klasse `datetime` stellt auch die Methode `timedelta` zur
Verfügung. Mit dieser Methode kann zu einem Datum eine Zeitspanne hinzugefügt
werden. Diese Zeitspanne kann in Tagen angegeben werden.

Die Syntax um einer Variablen ein Datum zuzuweisen lautet folgendermassen:

```python
geburtstag_mk = datetime.datetime(1969, 1, 23)
```

Der Wochentag eines Datums kann mit der Methode `weekday()` abgerufen werden. Im
Beispiel sieht das so aus:

```python
wochentag = geburtstag_mk.weekday()
print(wochentag) # 3 (23. Januar 1969 war ein Donnerstag)
```

Die Syntax, um einem gegebenen Datum `datum` eine Zeitspanne hinzuzufügen, lautet:

```python
datum = datum + datetime.timedelta(days=1)
```

### Beispiel für die Verwendung der Klasse `datetime`

Weisen Sie der Variablen `my_birthday` das Datum Ihres Geburtstags zu.
Kontrollieren Sie anschliessend mit der Methode `weekday()` den Wochentag Ihres Geburtstags.

In [ ]:
# vgl. oben

### `holidays` Library

Die Library `holidays` stellt unter anderem die Feiertage in der Schweiz zur
Verfügung. Dabei besteht die Möglichkeit, die Feiertage nach Kanton zu filtern.
Unter der entsprechnden Variablen werden die Feiertage in einer Datenstruktur
abgelegt, welche einem Python Dictinary ähnelt.

Eine Datenstruktur mit den Feiertagen für den Kanton Zürich wird mit folgender
Syntax angelegt:

```python
feiertage_zh = holidays.Switzerland(subdiv='ZH')
```

Um zu prüfen, ob ein Datum ein Feiertag ist, getestet werden, ob das Datum in
der Datenstruktur `feiertage_zh` enthalten ist. Dies geschieht mit der
Methode `in`:

```python
'2025-01-01' in feiertage_zh
```
Die Methode gibt `True` zurück, wenn das Datum ein Feiertag ist.

### Beispiel für die Verwendung der Library `holidays`

Erstellen Sie eine Datenstruktur mit den Feiertagen für den Kanton Zürich.
Prüfen Sie anschliessend, ob der 1. Mai 2025 ein Feiertag ist.

In [3]:
feiertage_zh = holidays.Switzerland(subdiv='ZH')
tag_der_arbeit = datetime.datetime(2023, 5, 1)
tag_der_arbeit in feiertage_zh

True

## Funktion zur Berechnung des Zustellungsdatums

Für die Berechnung des Datums der Zustellung werden die Sonn- und Feiertage
berücksichtigt. Zur Kontrolle der gesetzlichen Feiertage kann auf die Liste der 
[gesetzlichen
Feiertage](https://www.bj.admin.ch/dam/bj/de/data/publiservice/service/zivilprozessrecht/kant-feiertage.pdf.download.pdf/kant-feiertage.pdf)
des Bundesamtes für Justiz zurückgegriffen werden.

In [ ]:
def get_zustelldatum(abgabedatum: datetime.date) -> datetime.date:
    """
    Berechnet das Zustelldatum eines eingeschriebenen Briefes der
    Schweizerischen Post. 

    Diese Funktion kalkuliert, basierend auf dem Abgabedatum, das erwartete 
    Zustelldatum eines eingeschriebenen Briefes. Die Berechnung berücksichtigt:
    - Sonntage (keine Zustellung)
    - Feiertage im Kanton Zürich (keine Zustellung)
    - Standardzustellung am nächsten Werktag

    Args:
        abgabedatum (datetime.date): Das Datum, an dem der Brief bei der Post 
            aufgegeben wird.

    Returns:
        datetime.date: Das berechnete Zustelldatum. Falls das berechnete Datum
            auf einen Sonntag oder Feiertag fällt, wird das Datum auf den 
            nächsten Werktag verschoben.

    Raises:
        ValueError: Wenn das übergebene Datum kein gültiges datetime.date 
            Objekt ist.

    Example:
        >>> get_zustelldatum(datetime.date(2025, 4, 2))
        datetime.date(2025, 4, 3)
    """
    # Beginne mit dem Tag nach der Aufgabe (Standardzustellung am nächsten Werktag)
    datum = abgabedatum + datetime.timedelta(days=1)
    
    # Prüfe und überspringe Wochenenden (weekday() >= 5) und Feiertage
    # Die while-Schleife läuft so lange, bis ein gültiger Zustelltag gefunden wird
    while datum.weekday() >=5 or datum in holidays.CH(subdiv='ZH'):
        # Erhöhe das Datum um einen Tag, wenn der aktuelle Tag ein Sonntag
        # oder Feiertag ist
        datum += datetime.timedelta(days=1)
    
    return datum

## Funktion zur Berechnung der Kündigungsfrist

In [ ]:
def _set_kuendigungstermine(referenzdatum: datetime.date) -> list:
    """
    Berechnet die ortsüblichen Kündigungstermine für Mietwohnungen im Kanton Zürich.

    Diese Funktion ermittelt die nächstmöglichen Kündigungstermine basierend auf
    einem Referenzdatum. Im Kanton Zürich sind die ortsüblichen Kündigungstermine
    der 31. März, 30. Juni und 30. September.

    Args:
        referenzdatum (datetime.date): Das Datum, von dem aus die nächsten
            Kündigungstermine berechnet werden sollen.

    Returns:
        list: Eine Liste von datetime.date Objekten mit den nächsten
            Kündigungsterminen (31. März, 30. Juni, 30. September).
            Die Termine werden chronologisch sortiert zurückgegeben.

    Raises:
        ValueError: Wenn das übergebene Datum kein gültiges datetime.date 
            Objekt ist.

    Example:
        >>> _set_kuendigungstermine(datetime.date(2025, 4, 1))
        [datetime.date(2025, 6, 30), 
         datetime.date(2025, 9, 30), 
         datetime.date(2026, 3, 31)]
    """
    # Initialisiere leere Liste für die Kündigungstermine
    termine = []
    
    # Definition der Tage und Monate für die Kündigungstermine
    # 31. März, 30. Juni, 30. September
    days = [31, 30, 30]
    months = [3, 6, 9]
    
    # Bestimme das Jahr für die Kündigungstermine
    # Falls das Referenzdatum nach Juni liegt, werden die Termine für das
    # nächste Jahr berechnet
    if referenzdatum.month >= 6:
        year = referenzdatum.year + 1
    else:
        year = referenzdatum.year
    
    # Erstelle die Kündigungstermine durch Kombination von Jahr, Monat und Tag
    for i in range(3):
        termine.append(datetime.date(year, months[i], days[i]))
    
    return termine

In [ ]:
def get_kuendigungstermin(aufgabedatum: datetime.date) -> datetime.date:
    """
    Berechnet den nächstmöglichen Kündigungstermin für Mietwohnungen im Kanton Zürich.

    Diese Funktion ermittelt den nächstmöglichen Kündigungstermin basierend auf dem
    Aufgabedatum der Kündigung. Die Berechnung berücksichtigt:
    - Das Zustelldatum des Kündigungsschreibens (via get_zustelldatum)
    - Die gesetzliche Kündigungsfrist von 3 Monaten
    - Die ortsüblichen Kündigungstermine (31. März, 30. Juni, 30. September)

    Die Funktion verwendet zwei Hilfsfunktionen:
    - get_zustelldatum: Berechnet das Zustelldatum des Kündigungsschreibens
    - _set_kuendigungstermine: Ermittelt die möglichen Kündigungstermine

    Args:
        aufgabedatum (datetime.date): Das Datum, an dem die Kündigung bei der Post
            aufgegeben wird.

    Returns:
        datetime.date: Der nächstmögliche gültige Kündigungstermin. Dies ist
            entweder der 31. März, 30. Juni oder 30. September, je nachdem,
            welches Datum nach Ablauf der dreimonatigen Kündigungsfrist als
            nächstes folgt.

    Raises:
        ValueError: Wenn das übergebene Datum kein gültiges datetime.date 
            Objekt ist oder die Datumsberechnung ungültige Werte ergibt.

    Example:
        >>> get_kuendigungstermin(datetime.date(2025, 4, 1))
        datetime.date(2025, 9, 30)
    """
    # Berechne das Zustelldatum des Kündigungsschreibens
    zustellung = get_zustelldatum(aufgabedatum)
    
    # Hole die möglichen Kündigungstermine (31. März, 30. Juni, 30. September)
    kuendigungstermine = _set_kuendigungstermine(aufgabedatum)
    
    # Extrahiere Tag, Monat und Jahr aus dem Zustelldatum für die Berechnung
    # des provisorischen Kündigungstermins
    day = zustellung.day
    month = zustellung.month
    year = zustellung.year
    
    # Berechne den provisorischen Kündigungstermin (3 Monate nach Zustellung)
    termin = datetime.date(year, month + 3, day)
    
    # Ermittle den nächstmöglichen gültigen Kündigungstermin
    # Vergleiche den provisorischen Termin mit den ortsüblichen Terminen
    # und wähle den nächstfolgenden aus
    if termin < kuendigungstermine[0]:
        return kuendigungstermine[0]  # Erster möglicher Kündigungstermin
    elif termin < kuendigungstermine[1]:
        return kuendigungstermine[1]  # Zweiter möglicher Kündigungstermin
    else:
        return kuendigungstermine[2]  # Dritter möglicher Kündigungstermin

## Funktion zur Berechnung des Aufgabedatums

In [ ]:
def get_aufgabedatum(kuendigungstermin: datetime.date) -> datetime.date:
    """
    Berechnet das spätestmögliche Aufgabedatum für eine Kündigung nach Schweizer Mietrecht.

    Gemäss Schweizer Mietrecht muss die Kündigung dem Empfänger mindestens drei Monate
    vor dem Kündigungstermin zugestellt werden. Da die Zustellung per eingeschriebenem
    Brief typischerweise einen Tag dauert und die Zustellung an einem Werktag erfolgen
    muss, berechnet diese Funktion rückwärts vom Kündigungstermin das spätestmögliche
    Aufgabedatum.

    Parameter:
        kuendigungstermin (datetime.date): Der gewünschte Kündigungstermin des Mietverhältnisses

    Rückgabewert:
        datetime.date: Das spätestmögliche Datum für die Aufgabe der Kündigung unter
                      Berücksichtigung von Wochenenden und Feiertagen im Kanton Zürich

    Hinweise:
        - Die Funktion berücksichtigt Wochenenden und Feiertage im Kanton Zürich
        - Die gesetzliche Kündigungsfrist von drei Monaten wird eingehalten
        - Die Zustellung muss an einem Werktag möglich sein
    """
    # Berechne das Zustelldatum durch Abzug von 3 Monaten vom Kündigungstermin
    # Dies stellt die Einhaltung der gesetzlichen 3-monatigen Kündigungsfrist sicher
    month = kuendigungstermin.month - 3
    zustelldatum = datetime.datetime(kuendigungstermin.year, month, kuendigungstermin.day)
    
    # Verschiebe das Zustelldatum nach hinten, falls es auf Wochenenden (5=Samstag, 6=Sonntag)
    # oder Feiertage im Kanton Zürich fällt, um die Zustellbarkeit zu gewährleisten
    while zustelldatum.weekday() >= 5 or zustelldatum in holidays.CH(subdiv='ZH'):
        zustelldatum -= datetime.timedelta(days=1)
        
    # Berechne das Aufgabedatum (einen Tag vor Zustellung wegen Postlaufzeit)
    aufgabe = zustelldatum - datetime.timedelta(days=1)
    
    # Verschiebe das Aufgabedatum nach hinten, falls es auf Wochenenden oder Feiertage fällt
    # Dies stellt sicher, dass die Aufgabe an einem Werktag erfolgen kann
    while aufgabe.weekday() >= 5 or aufgabe in holidays.CH(subdiv='ZH'):
        aufgabe -= datetime.timedelta(days=1)
    
    return aufgabe